In [1]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np
import os


/home/simba9/miniconda3/envs/ultralytics/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [ ]:
# Загрузка модели и процессора CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


In [2]:


# Функция для подготовки изображений
def prepare_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    return inputs

# Функция для извлечения признаков изображений
def extract_image_features(image_path):
    inputs = prepare_image(image_path)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    return image_features.cpu().numpy()

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Путь к директории с изображениями
image_dir = "/home/simba9/pet/shov/dataset"
image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]


In [ ]:


# Извлечение признаков для всех изображений
image_features_list = [extract_image_features(img_path) for img_path in image_paths]
image_features_array = np.vstack(image_features_list)

# Кластеризация с использованием k-means
num_clusters = 5  # Укажите количество кластеров
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(image_features_array)

# Вывод результатов кластеризации
for img_path, cluster in zip(image_paths, clusters):
    print(f"Image: {img_path} is in cluster {cluster}")